In [ ]:
from scipy.linalg import norm
from scipy.spatial.transform import Rotation as R
from sklearn.neighbors import NearestNeighbors
import gzip
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

import glob
import os

In [ ]:
%run data_loaders/scannet_render_loader.py

In [ ]:
SCENE_NAME = 'scene0000_00'

In [ ]:
def get_sorted_scene_files(files):
    files = glob.glob(os.path.join('data', SCENE_NAME, files))
    files.sort()
    return files

In [ ]:
##-- Check Poses --##
pose_files = get_sorted_scene_files(os.path.join('pose', '*.txt'))

print('loaded', len(pose_files), 'poses')

In [ ]:
def get_train_val_split(pose_files, skip, max_index=len(pose_files), stride=1):
    pose_files = pose_files[0:max_index:stride]
    train_filenames = [pose_files[i] for i in range(len(pose_files)) if (i % skip) != 0]
    val_filenames = [pose_files[i] for i in range(len(pose_files)) if (i % skip) == 0]
    
    return train_filenames, val_filenames

In [ ]:
def load_poses(pose_files):
    rots = []
    ts = []
    for file in pose_files:
        with open(file) as csv_file:
            data = pd.read_csv(csv_file, delimiter=' ', index_col=None, header=None)
            
            rot = np.array(data.values[0:3,0:3])
            t = np.array(data.values[0:3,-1])
            rots.append(rot)
            ts.append(t)
    return rots, ts

In [ ]:
def get_nn_indices(neighbors, items):
    neigh = NearestNeighbors(n_neighbors=1)
    neigh.fit(neighbors)

    neigh_dist, neigh_index = neigh.kneighbors(items)
    
    return neigh_index

In [ ]:
def get_rotvecs_from_matrices(rots):
    rotvecs = []
    for rot in rots:
        rotvecs.append(R.from_matrix(rot).as_rotvec())
    
    return rotvecs

In [ ]:
def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))


#angles = []
#for i, val_rot in enumerate(val_rots):
#    angle = get_angle(val_rot, train_rots[nn_indices[i,0]])
#    angles.append(np.rad2deg(angle))
#
#print(angles)

In [ ]:
# Load train and validation poses
def get_val_nn_train_angles(train_rots, val_rots, unit='deg'):
    # Transform to axis representation
    train_rotvecs = get_rotvecs_from_matrices(train_rots)
    val_rotvecs = get_rotvecs_from_matrices(val_rots)

    # Find nearest neighbors by angle
    nn_indices = get_nn_indices(train_rotvecs, val_rotvecs)

    # Get angles to nearest neighbords in degrees
    angles = []
    for i, val_rotvec in enumerate(val_rotvecs):
        angle = angle_between(val_rotvec, train_rotvecs[nn_indices[i,0]])
        if unit == 'deg':
            angle = np.rad2deg(angle)
        angles.append(angle)
    
    return angles

In [ ]:
##-- Check Texture Coords --##
#uv_files = get_sorted_scene_files(os.path.join('uv', 'scene_nonbinary_vertexuv_flat_proj', '*'))
uv_files = glob.glob(os.path.join('..', 'texturemaprender', 'output', '*.gz'))
uv_files.sort()
print('loaded', len(uv_files), 'uv files')

In [ ]:
##-- Visualize Texture Coords --##
_SCREEN_HEIGHT, _SCREEN_WIDTH, _UV_CHANNELS = 968, 1296, 3

def load_texture_coord(file, compressed=True):
    if compressed:
        # Decompress texture coordinate file into a numpy array
        with gzip.open(file, 'rb') as f:
            uv_image = np.frombuffer(f.read(), dtype='float32')
            print('loading and decompressing')
    else:
        uv_image = np.fromfile(uv_image_path, dtype='float32')
        print('loading')    

    uv_image = np.reshape(uv_image, (_SCREEN_HEIGHT, _SCREEN_WIDTH, _UV_CHANNELS))
    uv_image = np.flip(uv_image, axis=0).copy()
    
    return uv_image
        
def visualize_texture_coord(uv_image):
    uv_color_image = np.zeros((_SCREEN_HEIGHT, _SCREEN_WIDTH, _UV_CHANNELS))
    uv_color_image[:,:,0:_UV_CHANNELS] = uv_image
    
    # Should assert that rows * cols == len(title) == len(display_images)
    plt.figure(figsize=(10, 10))
    plt.subplot(1, 1, 1)
    # getting the pixel values between [0, 1] to plot it.
    plt.imshow(uv_color_image)
    plt.show()

def visualize_texture_coord_mask(uv_image):
    uv_color_image = uv_image[:,:,-1]
    
    # Should assert that rows * cols == len(title) == len(display_images)
    plt.figure(figsize=(10, 10))
    plt.subplot(1, 1, 1)
    # getting the pixel values between [0, 1] to plot it.
    plt.imshow(uv_color_image, cmap='binary')
    plt.show()
    
def print_texture_coord_chunk(uv_image):
    print(uv_image[100:110, 1286:1296])

In [ ]:
uv_image = load_texture_coord(uv_files[0])
visualize_texture_coord(uv_image)
visualize_texture_coord_mask(uv_image)
#print_texture_coord_chunk(uv_image)

In [ ]:
# Display results
tv_files = [None]*4
tv_files[0] = get_train_val_split(pose_files, 6, 2700, 1)
tv_files[1] = get_train_val_split(pose_files, 6, 2700, 2)
tv_files[2] = get_train_val_split(pose_files, 6, 2700, 4)
tv_files[3] = get_train_val_split(pose_files, 6, 2700, 8)

title = 'Train:Val = {}:{} samples \n (min, max, mean) = ({:.1f}, {:.1f}, {:.1f})'

plt.figure(figsize=(15,4))
for i in range(len(tv_files)):
    train_files, val_files = tv_files[i]
    train_rots, _ = load_poses(train_files)
    val_rots, _ = load_poses(val_files)
    angles = get_val_nn_train_angles(train_rots, val_rots, unit='deg')
    
    #print('Min:', np.min(angles))
    #print('Max:', np.max(angles))
    #print('Mean:', np.mean(angles))
    
    plt.subplot(1, len(tv_files), i+1)
    plt.title(title.format(len(train_files), len(val_files), np.min(angles), np.max(angles), np.mean(angles)))
    num_bins = 20
    plt.hist(angles, bins=num_bins)
    plt.xlabel('degrees')
    plt.ylabel('count')
plt.show()

In [ ]:
#win = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]])
#filtered = signal.convolve(sig, win, mode='same') / sum(win)

loader = UVDataLoader('data', 1, True, 6, size=(1296, 968)).split_validation()#size=(256, 342))
for batch_idx, (data, target) in enumerate(loader):
    #for train_id in train_ids:    
        
    #sift = cv2.SIFT()
    detector = cv2.xfeatures2d.SIFT_create()
    #detector = cv2.FastFeatureDetector_create()
    #orb = cv2.ORB()
    #target = np.zeros((256, 256,3), np.uint8)
    original = (0.5 * target[0,:,:,:].permute(1, 2, 0).numpy() + 0.5) * 255
    original = original.astype(np.uint8)
    original = cv2.cvtColor(original, cv2.COLOR_RGB2BGR)
    original = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)

    blurred = cv2.blur(original.copy(), (3,3))

    original_g = cv2.cvtColor(original, cv2.COLOR_RGB2GRAY)
    blurred_g = cv2.cvtColor(blurred, cv2.COLOR_RGB2GRAY)

    kp_orig = detector.detect(original, None)
    kp_blur = detector.detect(blurred, None)

    print('Keypoint count:', len(kp_orig), len(kp_blur), len(kp_orig) / len(kp_blur))
    #print('Laplacian: ', cv2.Laplacian(original_g, cv2.CV_64F).var(), cv2.Laplacian(blurred_g, cv2.CV_64F).var())

    cv2.drawKeypoints(original_g, kp_orig, original)
    cv2.drawKeypoints(blurred_g, kp_blur, blurred)

    #lap = cv2.Laplacian(original_g, cv2.CV_64F).var()
    #if lap < 150:
    #    print('Laplacian:', lap)

    #filtered = signal.convolve(target, win, mode='same')

    plt.figure(figsize=(20,20))
    plt.subplot(1, 2, 1)
    plt.title('Target')
    plt.imshow(original)
    plt.subplot(1, 2, 2)
    plt.title('')
    plt.imshow(blurred)
    plt.axis('off')
        
    plt.show()
    
    #break